In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [39]:
#df1 = pd.read_csv('csvdata/output_Marktplatz-20180410-141306.csv')
df1 = pd.read_csv('csvdata/magentacloud/output_Marktplatz-20180412-111303.csv')
df1 = df1[df1['w']<100] # filter large boxes

In [40]:
df1['distance'] = df1.groupby(['track_id']).diff().dropna().apply(
    lambda row : np.sqrt((row['x']*.026)**2 + (row['y']*.037)**2), axis = 1)
df1['frame_diff'] = df1.groupby(['track_id'])[['frame_id']].diff()

In [41]:
df1['velocity'] = df1['distance'] / ( df1['frame_diff'] * (1.0/30))
df1['velocity_smooth'] = df1.groupby(['track_id'])['velocity'].apply(pd.rolling_mean, 5, min_periods=1) #Smoothing
df1['velocity_smooth'] = df1['velocity_smooth'] * 3.6
df1['velocity_smooth'] = np.clip(df1['velocity_smooth'], 0, 30)

C:\Users\Danda\Anaconda3\lib\site-packages\pandas\core\groupby.py:796: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,min_periods=1,center=False).mean()
  return func(g, *args, **kwargs)


In [42]:
#number of people per frame
counts = df1.groupby(['frame_id']).count().reset_index()[['frame_id', 'track_id']].rename({'track_id':'count'},axis=1)

In [43]:
df2 = pd.merge(df1,counts,on='frame_id')

In [44]:
df3 = df2.sort_values(['frame_id','velocity_smooth'], ascending=False).drop_duplicates(['frame_id']).sort_values('frame_id')
df3[df3.track_id == 1322]

,frame_id,track_id,x,y,w,h,distance,frame_diff,velocity,velocity_smooth,count
74646,3149,1322,1861,553,46,84,0.149586,1.0,4.487583,13.990058,27
74670,3150,1322,1852,554,43,78,0.236907,1.0,7.107215,18.545653,23
74694,3151,1322,1843,554,42,73,0.234000,1.0,7.020000,19.263399,25
74716,3152,1322,1839,553,43,73,0.110386,1.0,3.311571,19.782985,23
74865,3159,1322,1787,546,52,76,0.659410,1.0,19.782288,23.824473,19
74885,3160,1322,1766,504,79,119,1.647128,1.0,49.413852,56.645420,20
74904,3161,1322,1756,486,88,135,0.714952,1.0,21.448552,68.625297,19
74924,3162,1322,1748,480,92,140,0.304217,1.0,9.126511,72.812054,20
74944,3163,1322,1739,468,98,152,0.501888,1.0,15.056653,82.676056,21
74963,3164,1322,1736,465,98,155,0.135665,1.0,4.069951,71.363174,17


In [29]:
json = df2[['frame_id', 'track_id', 'velocity_smooth', 'count']].to_json(orient='records')

In [30]:
with open("json.txt", "w") as text_file:
    text_file.write(json)

In [31]:
#BoudingBox gesperrter Bereich
bb_x = 850
bb_y = 931
bb_w = 416
bb_h = 145

In [13]:
#df3

In [35]:
#df3['is_tabu'] = df1.
test = ((df1.x < (bb_x+bb_w)) & ((df1.x + df1.w) > bb_x) & (df1.y < (bb_y+bb_h)) & ((df1.y + df1.h) > bb_y))
df1['is_tabu'] = False 
df1.loc[test,'is_tabu'] = True


In [37]:
df1[df1.is_tabu == True]

,frame_id,track_id,x,y,w,h,distance,frame_diff,velocity,velocity_smooth,is_tabu
1288,44,11,1019,730,86,202,0.000000,1.0,0.000000,1.369599,True
1354,46,11,1019,730,85,202,0.045222,1.0,1.356650,1.600259,True
1386,47,11,1019,730,85,203,0.000000,1.0,0.000000,1.444259,True
2197,72,11,1022,733,82,200,0.000000,1.0,0.000000,1.332000,True
2226,73,11,1022,733,82,200,0.000000,1.0,0.000000,1.110000,True
2256,74,11,1023,733,81,200,0.026000,1.0,0.780000,1.044000,True
6517,211,131,1263,890,59,62,NaN,NaN,NaN,NaN,True
6545,212,131,1263,889,59,64,0.037000,1.0,1.110000,1.110000,True
6576,213,131,1263,889,59,64,0.000000,1.0,0.000000,0.555000,True
6607,214,131,1263,889,59,64,0.000000,1.0,0.000000,0.370000,True


In [46]:
np.clip(df1['velocity_smooth'], 0, 30)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
5               NaN
6               NaN
7               NaN
8               NaN
9               NaN
10              NaN
11              NaN
12              NaN
13              NaN
14              NaN
15              NaN
16              NaN
17              NaN
18              NaN
19              NaN
20              NaN
21              NaN
22              NaN
24              NaN
25              NaN
26              NaN
27         3.996000
28         3.996000
29         0.000000
30         4.883941
            ...    
141814    16.521113
141815     5.867777
141816     1.775988
141817     1.538388
141818     2.341024
141819     9.953713
141820     2.160000
141821     1.775988
141822     1.404000
141823     1.123200
141824     1.922400
141825     1.775988
141826     1.360800
141827     0.799200
141828     0.561600
141829     1.598400
141830     1.922400
141832     3.729565
141833     2.337588
